In [39]:
from IPython.display import JSON
from bs4 import BeautifulSoup
import pandas as pd
import requests
import API_keys
import datetime

In [4]:
def get_cities_dataframe():
    schema="gans"
    host="wbs-guns-project.cthpzqamxucy.us-east-2.rds.amazonaws.com"
    user="admin"
    password=API_keys.mysqlPassword
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    table_df = pd.read_sql_table(
    'cities',
    con=con
    )
    return table_df[:1]

In [22]:
def get_weather_tables(cities):
    today = datetime.date.today()
    tomorrow = today + datetime.timedelta(days=1)
    API_key = API_keys.OpenWeather_key
    city_weather_tables = pd.DataFrame()
    for index, row in cities.iterrows():
        url = (f"http://api.openweathermap.org/data/2.5/forecast?q={row['CityCountry']}&appid={API_key}&units=metric") 
        response = requests.get(url)
        json = response.json()

        dictionary = {"Date_Time":[],'Temperature':[],'Feels_like': [],'Humidity':[],'Weather':[],'Clouds':[],'Wind_speed':[],'Rain':[],'Snow':[]}

        for hour in json['list']:
            dictionary['Date_Time'].append(hour['dt_txt'])
            dictionary['Temperature'].append(hour['main']['temp'])
            dictionary['Feels_like'].append(hour['main']['feels_like'])
            dictionary['Humidity'].append(hour['main']['humidity'])
            dictionary['Weather'].append(hour['weather'][0]['description'])
            dictionary['Clouds'].append(hour['clouds']['all'])
            dictionary['Wind_speed'].append(hour['wind']['speed'])
            try:
                dictionary['Rain'].append(hour['rain']['3h'])
            except:
                dictionary['Rain'].append('0')
            try:
                dictionary['Snow'].append(hour['snow']['3h'])
            except:
                dictionary['Snow'].append('0')

        weather_table=pd.DataFrame.from_dict(dictionary)
        weather_table["Date_Time"] = pd.to_datetime(weather_table["Date_Time"])
        weather_table = weather_table[weather_table.Date_Time.dt.date == tomorrow]
        weather_table = weather_table.assign(CityCountry = row['CityCountry'])
        city_weather_tables = city_weather_tables.append(weather_table)
    return city_weather_tables

## def create_mysql_table(big_table):
    schema="gans"
    host="127.0.0.1"
    user="root"
    password=API_keys.mysqlPassword
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    big_table.to_sql('weather',con=con,if_exists='append',index=False)

In [38]:
create_mysql_table(get_weather_tables(get_cities_dataframe()))